In [ ]:
from inspect import Parameter, Signature
from typing import Annotated, get_type_hints

In [ ]:
{'properties': {
    'body': {
        'default': '',
        'description': 'Body of the issue',
        'examples': ['The code is not working',
                    'I would like to request '
                    'a new feature'],
        'title': 'Body',
        'type': 'string'},
    'owner': {'description': 'Owner of the '
                                'repository',
                'examples': ['openai', 'facebook'],
                'title': 'Owner',
                'type': 'string'},
    'repo': {'description': 'Name of the '
                            'repository',
                'examples': ['gpt-3', 'react'],
                'title': 'Repo',
                'type': 'string'},
    'title': {'description': 'Title of the issue',
                'examples': ['Bug in the code',
                            'Feature request'],
                'title': 'Title',
                'type': 'string'}},
'required': ['owner', 'repo', 'title'],
'title': 'CreateIssueRequest',
'type': 'object'}

In [ ]:
schema_type_python_type_dict = {
    'string': str,
    'number': float,
    'boolean': bool,
    # 'array': list,
    # 'object': dict
}

fallback_values = {
    'string': "",
    'number': 0.0,
    'boolean': False,
    'object': {},
    'array': []
}
def pydantic_model_from_param_schema(param_title, param_schema):
    fields = {}
    for prop_name, prop_schema in param_schema.items():

        prop_name: (str, Field(..., title=prop_info['title'], default=prop_info.get('default')))
    if prop_name in required else
    (str, Field(None, title=prop_info['title'], default=prop_info.get('default')))


    

def get_signature_format_from_schema_params(
        schema_params
):
    parameters = []

    for param_name, param_schema in schema_params['properties'].items():
        param_type = param_schema['type']
        param_title = param_schema['title'].replace(" ", "")
        required_params = param_schema.get('required', [])

        if param_type in schema_type_python_type_dict:
            signature_param_type = schema_type_python_type_dict[param_type]
        else:
            signature_param_type = get_signature_format_from_schema_params(param_title,
                                                                           param_schema,
                                                                           required_params)

        # param_type = schema_type_python_type_dict[param_schema['type']]
        param_name = param_schema['name']
        param_default = param_schema.get('default', fallback_values[param_type])
        param_annotation = Annotated[signature_param_type, param_schema['description']]
        param = Parameter(
            name=param_name,
            kind=Parameter.POSITIONAL_OR_KEYWORD,
            annotation=param_annotation,
            default=param_default
        )
        parameters.append(param)


        

    

In [ ]:
import types

# Not supporting posiotonal arguments intentionally
# demo_func = lambda *args, **kwargs: print(args, kwargs)

demo_func = lambda *args, **kwargs: print(args, kwargs)

parameters = [
    Parameter(
        name="base", 
        kind=Parameter.POSITIONAL_OR_KEYWORD, 
        annotation=Annotated[str, "Base currency: amount and currency symbol"], 
        # default=Parameter.empty if param in required_params else Parameter.default
    ),
    Parameter(
        name="quote_currency", 
        kind=Parameter.POSITIONAL_OR_KEYWORD, 
        annotation=Annotated[str, "Quote currency symbol"], 
        # default=Parameter.empty if param in required_params else Parameter.default
    ),
]
new_sig = Signature(parameters, return_annotation=str)

func = types.FunctionType(demo_func.__code__, 
                          globals=globals(), 
                          name="currency_calculator2", 
                        #   closure=template_function.__closure__
                          )
func.__signature__ = new_sig
func


In [ ]:
func(3, 5)

In [ ]:
inspect.signature(func)

In [ ]:
import inspect
from pydantic import BaseModel, Field
from typing_extensions import Annotated

In [ ]:
def currency_calculator(
    base: Annotated[str, "Base currency: amount and currency symbol"],
    quote_currency: Annotated[str, "Quote currency symbol"] = "USD",
) -> str:
    quote_amount = base + " " + quote_currency
    return quote_amount

currency_calculator

In [ ]:
currency_calculator.__closure__

In [ ]:
inspect.signature(currency_calculator)

In [ ]:
composio.autogen.register.slack()
composio.autogen.register.hubspot()

composio.autogen.register(
    [composio.slack.send_message.args(
            caller=chatbot,
            executor=user_proxy,
    ),
     composio.slack.block_one],
    caller=chatbot,
    executor=user_proxy,
)


autogen.agentchat.register_function(
    composio.slack.send_message,
    caller=chatbot,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)

In [1]:
from autogen import AssistantAgent, UserProxyAgent
from composio_autogen import ComposioToolset, App, Action
import os
import dotenv

dotenv.load_dotenv()

llm_config = {"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]}

chatbot = AssistantAgent(
    "chatbot",
    system_message="Reply TERMINATE when the task is done or when user's content is empty",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    "user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and "TERMINATE" in x.get("content", ""),
    human_input_mode="NEVER", # Don't take input from User
    code_execution_config = {"use_docker": False}
)


In [2]:
# Import from composio_autogen
from composio_autogen import ComposioToolset, App, Action

# Initialise the Composio Tool Set
composio_tools = ComposioToolset()

# Register the preferred Applications, with right executor. 
composio_tools.register_tools(tools=[App.GITHUB],caller=chatbot, executor=user_proxy)
